# Source: Rivals

In [1]:
#hide
import core_constants as cc
import functions as fx
import json
import pandas as pd
import sqlite3 as sql

## Set Notebook Settings

In [ ]:
conference = 'sunbelt'

years = cc.get_defYears()
headers= cc.get_header()
schoolsList = cc.get_schoolsList()
teamDirectory = cc.get_htmlDir('rivals', conference, 'teams')
playerDirectory = cc.get_htmlDir('rivals', conference, 'recruits')
dataset = 'Rivals'
#testDirectory = '..//tests//'

## Get & Save the Teams & Players Page HTML
#### Source: https://maryland.rivals.com/commitments/football/2012
> This page contains metadata of each player along with the Rivals ranking and stars.  Unlike 247Sports, we process the fetch and save of both pages directly from a single function

In [ ]:
fx.get_Rivals(conference, schoolsList, years, headers, sleepyTime=6)

## Process Local Rivals HTML Files

> All of this processing is done locally, using the files saved in the previous few steps.  This creates an exhaustive store of all the fields grabbed from the scrapes.

>WARN: There is a different process_rivals scrip tin the functions.py file, and it probably needs to be integrated into process_Rivals before you re-run this.

In [ ]:
cc.save_records('scrapedData', 'rivals_' + conference, fx.process_Rivals(playerDirectory, conference, schoolsList))

## Clear DB
> Useful for a clean start.  This removes all of the records for this dataset from the following structures: SourcedPlayers, RecordLinks.  All of the Views auto-cleanse themselves.

In [ ]:
fx.clearDB(dataset)

## Save to Database

In [2]:
fx.toDB_Rivals()

'DB Write is done'

## Strict Matching
> This saves it to RecordLinking where ID == ID, but returns IDYR as the matching target

In [ ]:
fx.literalLinking(dataset)

## Fuzzy Matching w/ Threshold

> This is automatically pushing fuzzy matches above a certain threshold into the DB without the need for review [last part isn't true!  not automatically writing currently]

In [ ]:
fuzzyDF = fx.doFuzzyMatching(dataset, 'Sports247')

## Create the Annotation File

> This changes the dataframe into a MultiIndex data frame that the annotation function requires

In [ ]:
conn = sql.connect(cc.databaseName) 
          
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM SourcedPlayers
                               WHERE KeyDataSet = 1
                               ''', conn)

df_247 = pd.DataFrame(sql_query, columns = ['IDYR', 'College', 'Year', 'Position'])
df_247.set_index('IDYR', append=False, inplace=True)
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM UnlinkedNCAA
                               ''', conn)

df_NCAA = pd.DataFrame(sql_query, columns = ['ID', 'College'])
df_NCAA.set_index('ID', append=False, inplace=True)

fuzzyMI = pd.MultiIndex.from_frame(fuzzyDF)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_ncaa2.json",
    fuzzyMI[0:300],
    df_NCAA,
    df_247,
    dataset_a_name="NCAA",
    dataset_b_name="Master"
)

## Read in the Annotation File
> Take the resulting Annotation file after handling the processing and insert it into the right table

In [ ]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//ncaa_results.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

## Insert Annotations to RecordLinks

In [ ]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 5, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()